# Install necessarily library


In [ ]:
!pip install transformers datasets tensorflow
!pip install pandas
!pip install gdown
!pip install rarfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


# Import all the library needed in here
we will use Tensorflow library

In [ ]:
import pandas as pd
import numpy as np
import gdown
import rarfile
import tensorflow as tf
import pickle
from datasets import Dataset
from transformers import T5Tokenizer, TFAutoModelForSeq2SeqLM, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split

In [ ]:
def download_file_from_google_drive(file_id, dest_path):
    gdown.download(f"https://drive.google.com/uc?id=1fFVcdw8S8wJds8yX18FYu7n_z646AvKl", dest_path, quiet=False)

In [ ]:
file_id = '1NPTZM9MhW0R7-fb_yeuTK8bsYz86Q0N9'
dest_path = 'dataset.rar'
download_file_from_google_drive(file_id, dest_path)

Downloading...
From (original): https://drive.google.com/uc?id=1fFVcdw8S8wJds8yX18FYu7n_z646AvKl
From (redirected): https://drive.google.com/uc?id=1fFVcdw8S8wJds8yX18FYu7n_z646AvKl&confirm=t&uuid=7ffb4380-caf8-4964-81e2-5446e5deb1c8
To: /content/dataset.rar
100%|██████████| 128M/128M [00:03<00:00, 37.9MB/s]


In [ ]:
with rarfile.RarFile(dest_path) as rf:
    rf.extractall('./dataset')

In [ ]:
train_df = pd.read_csv('dataset/train_data.csv')

train_df = train_df[:int(len(train_df)*0.5)]

test_df = pd.read_csv('dataset/test_data.csv')

test_df = test_df[:int(len(test_df)*0.5)]

print(f'test data length is : {len(test_df)}')
print(f'train data length is : {len(train_df)}')

test data length is : 9387
train data length is : 35676


In [ ]:
def filtered_data_1500(data):
  return data[data['news_text'].str.len() <= 1500]

# Filter out rows where news_text length is more than 1500 characters
test_df = filtered_data_1500(test_df)
train_df = filtered_data_1500(train_df)

# Print the number of remaining rows
print(len(test_df))
print(len(train_df))

2128
7959


In [ ]:
# Find the longest news_text
longest_news_text = test_df['news_text'].apply(len).idxmax()

# Find the longest summary_text
longest_summary_text = test_df['summary_text'].apply(len).idxmax()

# Retrieve the longest news_text
longest_news_text_value = test_df.loc[longest_news_text, 'news_text']
# Retrieve the longest news_text
longest_summary_text_value = test_df.loc[longest_summary_text, 'summary_text']

print(f"The longest news_text is: {longest_news_text_value}")
print(len(longest_news_text_value))
print(len(longest_summary_text_value))

The longest news_text is: Jakarta , CNN Indonesia - - Ketua Umum Partai Amanat Nasional Zulkifli Hasan menilai kasus dugaan korupsi proyek KTP elektronik yang menyeret puluhan nama anggota DPR seharusnya tidak bergulir ke penggunaan hak angket DPR . Ia berkata , DPR seharusnya menyerahkan pengusutan perkara tersebut kepada KPK . “ Ini kan urusan KPK , masa mau ditarik - tarik ke angket . Jadi biarlah yang soal maling , KPK yang urus , ” ujar Zulkifli di Jakarta , Senin ( 13 / 3 ) . Zulkifli menyebut dugaan korupsi proyek e - kTP merupakan pengkhianatan yang dilakukan pejabat terhadap rakyat dan negara . Apalagi , kata dia , perkara itu menyangkut anggaran negara senilai Rp 5,9 triliun . Zulkifli   yakin , uang panas proyek e - KPT dinikmati banyak pihak , baik dari unsur legislatif , eksekutif dan swasta . “ Ini mega skandal dan mega korupsi . Saya dukung KPK seribu persen untuk tuntaskan kasus ini , ” ujarnya . Terkait adanya politikus PAN yang disebut jaksa menerima uang proyek e - K

In [ ]:
# Reset the index and drop the old index
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
train_df['news_text'][0]

'Merdeka.com - Indonesia Corruption Watch ( ICW ) meminta Komisi Pemberantas   Korupsi ( KPK ) ikut memantau perkembangan atas meninggalnya saksi kunci kasus mega korupsi e - KTP , Johannes Marliem . Peneliti ICW Divisi Hukum dan Monitoring Peradila , Aradila Caesar mengatakan momentum meninggalnya saksi kunci tersebut menimbulkan kejanggalan dan tanda tanya besar . " Orang meninggal kita kan tidak bisa prediksi itu bukan kuasa kita . Tapi kalau kita melihat momentum kan ada suatu kejanggalan . Kenapa momentum meninggalnya , saat kasus e - ktp sedang ditangani oleh KPK , " katanya seusai konferensi pers di Kantor Sekeretariatan ICW ,   Jakarta , Minggu(13 / 8 ) . Pihak ICW meminta KPK turut menyelidiki kematian saksi kunci ini dan menjelaskan kepada masyarakat apakah ada keterkaitan dengan permasalahan korupsi e - KTP atau hal-hal lain dibalik kematian Johannes . " Kita minta KPK dan juga bekerja sama dengan pihak otoritas untuk menyelidiki kematian dari saksi kunci tersebut dengan ser

In [ ]:
dataset_test = Dataset.from_pandas(test_df)
dataset_train = Dataset.from_pandas(train_df)

# Load tokenizer transformer model from Hugging Face

In [ ]:
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def preprocess_function(examples):
    inputs = examples['news_text']
    targets = examples['summary_text']

    model_inputs = tokenizer(inputs, max_length=1500, truncation=True, padding='max_length')

    # Tokenize targets
    labels = tokenizer(text_target=targets, max_length=600, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]

    # Prepare decoder input IDs
    decoder_input_ids = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs


In [ ]:
# Apply the preprocessing function to the entire dataset
tokenized_datasets = dataset_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/7959 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/2128 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['decoder_input_ids'][0]

[100,
 15,
 28662,
 18436,
 15,
 4,
 10763,
 745,
 15,
 5,
 928,
 8840,
 3082,
 839,
 13,
 8302,
 21832,
 15,
 4,
 6971,
 15,
 5,
 2966,
 2558,
 918,
 107,
 1282,
 26,
 3204,
 5171,
 3745,
 5415,
 8804,
 867,
 15,
 7,
 313,
 4776,
 27,
 22771,
 1853,
 440,
 2381,
 16,
 1193,
 28,
 158,
 1296,
 64,
 11197,
 13685,
 36,
 22,
 4011,
 8804,
 867,
 15,
 7,
 313,
 4776,
 46,
 539,
 7,
 2209,
 74,
 18,
 10781,
 1081,
 22771,
 15,
 3,
 26575,
 10763,
 745,
 26141,
 4281,
 16,
 28076,
 155,
 973,
 13,
 7037,
 13,
 27,
 1170,
 13,
 7037,
 13,
 18019,
 796,
 8962,
 1282,
 26,
 3204,
 5171,
 65,
 1834,
 30,
 3625,
 4832,
 36,
 16,
 1678,
 6206,
 146,
 15,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
def dataset_to_tf(dataset):
    def gen():
        for ex in dataset:
            yield (
                {
                    'input_ids': ex['input_ids'],
                    'attention_mask': ex['attention_mask'],
                    'decoder_input_ids': ex['decoder_input_ids']
                },
                ex['labels']
            )
    return tf.data.Dataset.from_generator(
        gen,
        (
            {
                'input_ids': tf.int32,
                'attention_mask': tf.int32,
                'decoder_input_ids': tf.int32
            },
            tf.int32
        ),
        (
            {
                'input_ids': tf.TensorShape([None]),
                'attention_mask': tf.TensorShape([None]),
                'decoder_input_ids': tf.TensorShape([None])
            },
            tf.TensorShape([None])
        )
    )

train_dataset = dataset_to_tf(tokenized_datasets)
test_dataset = dataset_to_tf(tokenized_test)

batch_size = 1
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [ ]:
# Load the model
model = TFT5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

tf_model.h5:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at cahya/t5-base-indonesian-summarization-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Define optimizer with a learning rate schedule
learning_rate = 5e-5
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=1000,
    end_learning_rate=0.0
)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
)

# Compile model
model.compile(optimizer=optimizer)

In [ ]:
num_train_epochs = 1
steps_per_epoch = len(tokenized_datasets) // batch_size // 16  # Adjust for gradient accumulation

model.fit(
    train_dataset,
    epochs=num_train_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=500 // batch_size,  # Number of validation steps
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir='./logs', update_freq=10),
        tf.keras.callbacks.ModelCheckpoint(filepath='./transfer_pegasus', save_freq='epoch')
    ]
)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
497/497 [==============================] - 857s 1s/step - loss: 0.6643 - val_loss: 0.0267


In [ ]:
# Tokenize input text
input_text = """
Yoga adalah seorang cendikiawan dari universitas teknologi Bandung. Dia menjadi populer karena kepintarannya . Karena itu banyak gadis birahi kepadanya. Kadang kating juga sering berpaling dari pacarnya karena ke gagahan yang di berikan yoga. Sekarang yoga adalah seorang pengusaha kelas atas. Banyak cindo yang berinvestasi kepadanya. Ada 10 cindo yang ingin menjadi mitra kedepan..dalam mengurus hal ini. Yoga memiliki ajudan sejati bernama Mama's Arifin. Mama's Arifin ini adalah seorang ajudan yang tampan dan rupawan. Terkadang mamas Arifin suka sekali membantu bosnya dalam mengurus pekerjaan.. setiap hari Senin mamas Arifin memijati bos yoga. Dan setiap hari selasa.mamas Arifin mencarikan sebuah ak47 berkaliber bold untuk digunakan bosnya berburu ubur ubur
"""
tokenized_input = tokenizer(input_text, return_tensors="tf")

# Generate summary
generated_summary_ids = model.generate(
    input_ids=tokenized_input["input_ids"],
    max_length=500,  # Adjust as needed
    num_beams=4,  # Adjust as needed
    early_stopping=True
)

# Decode generated summary
generated_summary = tokenizer.decode(generated_summary_ids[0], skip_special_tokens=True)

# Print generated summary
print("Generated Summary:")
print(generated_summary)

Generated Summary:



In [ ]:
print(generated_summary)

In [ ]:
model.save_pretrained("textSum_model2")

tokenizer.save_pretrained('textSum_tokenizer2')

('textSum_tokenizer2/tokenizer_config.json',
 'textSum_tokenizer2/special_tokens_map.json',
 'textSum_tokenizer2/spiece.model',
 'textSum_tokenizer2/added_tokens.json')

In [ ]:
import shutil

# Compress the tokenizer directory
shutil.make_archive('textSum_model', 'zip', 'textSum_model')

shutil.make_archive('textSum_tokenizer', 'zip', 'textSum_tokenizer')


'/content/textSum_tokenizer.zip'